In [ ]:
# 13.07.23 BG - Option STC - Optionsergebnis korrigiert 
# 11.07.23 BG - Neues Feld fxRateToBase aus Trades mit übertragen 
# 09.07.23 BG - Richtiges Umsetzung von "Assignments" 
# 09.07.23 BG - erste Schritte zu MS-Money-Export 
# 08.07.23 BG - Update des Statments , Uniue Key + levelOfDetail 
# 07.07.23 BG - Jede Bewegung, auch wenn 0 Steuer, kommt in die Tax-Tabelle 
# 06.07.23 BG - Balance noch mit ausgegeben 
# 06.07.23 BG - Nur SBUX zur Fehlersuche 
# 06.07.23 BG - amount umgestellt auf BaseCurreny currency ergänzt 
# 05.07.23 BG - umgestellt auf "levelOfDetail" 
# 05.07.23 BG - Abklemmen der Offenen Positionen 
# 05.07.23 BG - Handel ohne Expiry geklärt (= 0 DTE) 
# 05.07.23 BG - Combo-Beschriftung korrigiert 
# 04.07.23 BG - die US-Zeilen werden aus gefilfter (aber bei Money sollen sie auftauchen)
# 02.07.23 BG - Jetzt geht es los, stimmt der Datensatz 23 ? 
# 02.07.23 BG - Close oh23 Positionen - klappt 
# 01.07.23 BG - Idee von Frank:  wir lassen die Statements/Trades, die nicht im virtuellen Depot sind, aus
#                    un markieren diese in der Tabelle "statements", neues Feld txInfo als .. ausgelassen 
# 30.06.23 Bg - weiter Fehler Suche 
# 29.06.23 BG - Daten ohne Positionen / erste Versuche 
# 28.06.23 BG - Mit Fehlerabfrage, wenn es keinen Trade gibt 
# 28.06.23 BG - Stand auf TeFiX - IB Tax Info 
# 28.06.23 BG - STK testweise eingebaut 
# 27.06.23 Bg - Verlust mit - Zeichen 
# 27.06.23 BG - fifo fertig gestellt 
# 26.06.23 BG - fiFo Result für die Steuer ermitteln 
# 25.06.23 BG - erste Ansätze zu SteuerStatements 
# 23.06.23 BG - Umbau auf die neuen Namesgebung der Datenbank + Feldnamen
# 18.06.23 BG - Depot - opopopopopCombo - Erkennung 
# 17.06.23 BG - erster Wurf
#
# Strategie: lese Statmenments / trades ein
#  
# 18.06. - Offene Themen:   # !!!!!! 3/4 -> Combos noch nicht gelöst !!!!! Iron Condor / Butterfly 
# Abstand der Strikes !!! 
# Nettoprämie ermitteln !!!  
# Zuteilung richtig machen !!
#
# 1,1 - Open-Order - in das Depot eintragen (incl. Quantity)  
# Close-Order - 
#  2.1     suche die passende Open-Order
#  2.2     ermittle den Gewinn  
#  2.3     Trage Closed ein
#  2.4     Update die Quantiy, falls 0 lösche den Eintrag im Depot 
#
#
#
#

from datetime import datetime
from decimal import Decimal # 26.06.
import pyodbc  # Datenbankanbindung an Access 

# from quiffen import Qif, QifDataType # 09.07.23 BG - MS-Money Import 
# import quiffen Zu Kompliziert, 



# ---------- Utilities -------------
def calculateDaysToExpiration( trade ):
    print (f"+++ 123 calculatedaysToExpiration- {trade}" ) 
    print (f"+++ 123 calculatedaysToExpiration- Expiry-Date: {trade.expiryDate} - TradeDate: {trade.tradeDate} ")  
    # 05.07.23 z.B. beim Aktienkauf gibt es kein expiryDate 
    if trade.expiryDate == None:
        return 0
    else:   
        daysDelta = trade.expiryDate - trade.tradeDate    
        days = daysDelta.days 
        return days   

def calculateDaysInTrade ( openTrade, trade ):
    print ( openTrade )
    opTradeDate = openTrade.tradeDate
    tradeDate = trade.tradeDate
    print ( f"+++ 128 - calculateDaysInTrade  {opTradeDate}   {tradeDate}")
    daysDelta = trade.tradeDate - openTrade.tradeDate
    return daysDelta.days 

def tradetimeEqual ( trade , lastTrade ) :  
    if lastTrade == None :
        return False
    if trade.IBDateTime == lastTrade.IBDateTime : 
        return True
    else :
        return False
    
def calculateCombo ( dbCursor, statement, trade, lastTrade ):
    if tradetimeEqual (trade, lastTrade ):
        # print (f"+++ 145 calculateCombo ID: {statement.id}")
        # print ( statement ) 
        # print (trade)
        # print (lastTrade) 
        opCombo = f"Combo-{statement.id}"  # Korigiert am 05.07. BG 
        # BPS oder BCS berechnen 
        if ( trade.quantity > 0 ):
            if ( trade.strike < lastTrade.strike):
                zusatz = "BullPS"
            else:
                zusatz = "BullCS"
        else: 
            if ( trade.strike > lastTrade.strike):
                zusatz = "BearPS"
            else:
                zusatz = "BearCS"
        opCombo = zusatz + "-" + opCombo 

        # !!!!!! 3/4 -> Combos noch nicht gelöst !!!!!

        # dann muss der Vorgängerdatensatz auch ein Combo bekommen 
        updateSql = f"UPDATE openPositions SET combo = '{opCombo}'  WHERE transactionID = {lastTrade.transactionID} "
        print ( f"+++ Combo: Update-SQl {updateSql}" )
        dbCursor.execute ( updateSql )
        
    else:
        opCombo = ""
    #print (f"+++ 172 - Combo erzeugen - {combo}")
    return opCombo 

def selectStatement ( dbCursor, transactionID):
    sql = f"SELECT * FROM statements WHERE transactionID = {transactionID}"
    print ( "++ Select Statment " + sql )
    dbCursor.execute ( sql )
    statement = dbCursor.fetchone () 
    if statement is None: # 28.06.23 Fehlrabfrage 
        print  (f"+++ 172 - selectStatement - Kein Statement gefunden für TransaktionsID: {transactionID}")
    return statement 

def updateStatementTxInfo ( dbCursor, statement, txInfo ):
    #sql = f"UPDATE statements SET txInfo = '{txInfo}'    WHERE transactionID = {transactionID} "
    #print ( "+++ 177 -  updateStatment tx Info ", sql)
    #dbCursor.execute ( sql )
    #dbCursor.commit()
    updateStatementField ( dbCursor, statement, txInfo , "txInfo" )


def updateStatementOpInfo ( dbCursor, statement, opInfo ):
    #sql = f"UPDATE statements SET opInfo = '{opInfo}'    WHERE transactionID = {statement.transactionID} AND levelOfDetail = {statement.levelOfDetail}"
    #print ( "+++ 177 -  updateStatment op Info ", sql)
    #dbCursor.execute ( sql )
    #dbCursor.commit()
    updateStatementField ( dbCursor, statement, opInfo , "opInfo" )

def updateStatementField ( dbCursor, statement, str , field ):
    # 08.07.23 BG don´t repeat yourself 
    # 08.07.23 BG Beim Update muss .. levelOfDetail dazu, weil es zwei Datensaetze mit der selben ID gibt  
    sql =  f"UPDATE statements SET {field} = '{str}'    "
    sql += f" WHERE transactionID = {statement.transactionID} AND "
    sql += f" levelOfDetail = '{statement.levelOfDetail}' "
    print ( "+++ 217 -  updateStatmentField op Info ", sql)
    dbCursor.execute ( sql )
    dbCursor.commit()

def selectTrade ( dbCursor, transactionID):
    sql = f"SELECT * FROM trades WHERE transactionID = {transactionID}"
    print ( "+++ 191 - Select Trade " + sql )
    dbCursor.execute ( sql )
    trade = dbCursor.fetchone ()  
    if trade is None:
        print  (f"+++ 187 - selectTrade -Kein Trade gefunden für TransaktionsID: {transactionID}")
    return trade 

def selectOpenTransactionID ( dbCursor, trade ):
    # Suche die älteste Position zu dieser conid (statt Beschreibung)  
    sql = f"SELECT MIN(transactionID) FROM openPositions WHERE conid = {trade.conid} " 
    print ( "++ selectOpenTrade " + sql )
    dbCursor.execute ( sql )
    openTransactionID = dbCursor.fetchone ()[0] 
    print (f"+++ - 196 - selectOpenTransactionID SEL: conid = {trade.conid} gefundene Transactions ID: {openTransactionID}")
    return openTransactionID 

def opGetPositionsCount (dbCursor, conid):
    sql = f"SELECT count(*) AS anzahl FROM openPositions WHERE conid = {conid}"
    print ( "+++ opGetPositionCount" + sql )
    dbCursor.execute ( sql) 
    zeile = dbCursor.fetchone ()
    print ( zeile )
    anzahl = zeile.anzahl 
    print (f"+++ - 206 - GetPositionsCount WHERE conid = {conid} --> Anzahl {anzahl} ")
    return anzahl 

#-----------------------------------------------
#
# MS-Money / QIF -   09.07.23 - Neustart
#  #
#qif = quiffen.Qif()  
qifBaseCurrency = None 
qifCurrency = None
qifBaseCurrencyAccount = None 
qifCurrencyAccount = None

#    assert transaction.date == datetime(2022, 2, 1)
#    assert transaction.amount == 100
#    assert transaction.check_number == 1
#    assert transaction.payee == "Test Payee"
#    assert transaction.memo == "Test Memo"
#   assert transaction.payee_address == "Test Address"
#    assert transaction.category == Category(name="Test Category")
#    assert transaction.splits == [Split(amount=100)]

def qifInsertTransaction ( statement):
    global qif, qifBaseCurrencyAccount, qifCurrencyAccount
    levelOfDetail = statement.levelOfDetail
    if levelOfDetail == "BaseCurrency":
        if qifBaseCurrency == None:
            a = 1 
            # Ersten Eintrag machen Konto beginnt 
 #           qifBaseCurrencyAccount = quiffen.Account(name='BaseCurrency', desc='BasisAccount', balance = statement.balance)
            #qifBaseCurrencyAccount.balance = statement.balance
  #      tr = quiffen.Transaction ( date = statement.datum , amount = statement.amount,  
   #                         memo = statement.activityDescription, 
    #                        # category = dict(cat="kat", sub="subkategorie"),
     #                       payee = statement.description  )
      #  qifBaseCurrencyAccount.set_header(quiffen.AccountType.BANK)
       # qifBaseCurrencyAccount.add_transaction ( tr ) 
        # qifBaseCurrencyAccount.transactions 
    
    
    if levelOfDetail == "Currency":
        #if qifBaseCurrency == None:
        #    qifCurrencyAccount = quiffen.Account(name='Currency', desc='Account', balance = statement.balance)
            qifBaseCurrencyAccount.balance = statement.balance
        # cat = {cat="kat", sub="subkategorie"}
        #tr = quiffen.Transaction ( date = statement.datum , amount = statement.amount, 
         #                   memo = statement.activityDescription, 
          #                  # category = dict(cat="kat", sub="subkategorie"),
           #                 payee = statement.description )
        #qifCurrencyAccount.set_header(quiffen.AccountType.BANK)
        #qifCurrencyAccount.add_transaction ( tr ) 

#------------ Tax - Utilities ---------------------
# 06.07.23 BG - Neu 
def updateTxAmountCurrency ( dbCursor, statement ):  
    #sql = f"UPDATE statements SET IBcurrency = '{statement.IBcurrency}', amount = {statement.amount}    " 
    #sql += f"WHERE transactionID = {statement.transactionID} "
    #p#rint ( "+++ 238 -  updateTX 2. Währung ", sql)
    
    sql = "UPDATE taxstatements SET IBcurrency = ?, amount = ? , balance = ? WHERE transactionID =  ?  " 
    werte = ( statement.IBcurrency , statement.amount , statement.balance , statement.transactionID)
    print ( f"+++ 238 -  NEU : ID {statement.id} updateTX 2. Währung ", sql, werte )
    #dbCursor.execute ( sql , werte )
    #dbCursor.commit()
    
    # Verzweifelt, keien bessere Lösung 
    id = statement.transactionID
    updateTxField ( dbCursor , "IBcurrency", statement.IBcurrency, id , "S")
    updateTxField ( dbCursor , "amount", statement.amount, id , "I")
    updateTxField ( dbCursor , "balance", statement.balance, id , "I")
    
def updateTxField ( dbCursor , field, value, id , typ):
    if typ == "S" : # String 
        trenner = "'"
    else:
        trenner = " "
    sql = f"UPDATE taxstatements SET {field} = {trenner}{value}{trenner} WHERE transactionID = {id}"
    print ( f"+++ 255 -  NEU : Feldweise {sql}" )
    # print ( "Type von ", type(sql))
    dbCursor.execute ( sql )
    dbCursor.commit()


#------- ---------- Open Positions Utilities -------------------
def opSelectTrade ( dbCursor, transactionID):
    sql = f"SELECT * FROM openPositions WHERE transactionID = {transactionID}"
    dbCursor.execute ( sql )
    return dbCursor.fetchone ()  

def opDeleteTrade ( dbCursor, opTrade ):
    transactionID = opTrade.transactionID 
    sql = f"DELETE FROM openPositions WHERE transactionID = {transactionID} "
    print ( f"+++ 218 opDeleteTrade {sql}")
    dbCursor.execute ( sql )
    dbCursor.commit()

def opUpdateTrade ( dbCursor, opTrade, openStatement, statement, trade ):
    tradeQuantity = trade.quantity
    opTradeQuantity = opTrade.quantity
    newQuantity =  opTradeQuantity + tradeQuantity 
    newAmount = openStatement.amount + statement.amount 
    transactionID = opTrade.transactionID
    sql = f"UPDATE openPositions SET quantity = {newQuantity}, amount = {newAmount}    WHERE transactionID = {transactionID} "
    print ( f"+++ 228 - {sql}")
    dbCursor.execute ( sql )
    dbCursor.commit()

# ------------ Hauptbestandteile ----------------------
def updateOpenPositions ( dbCursor, statement, trade, lastTrade ): # lastTradeTime, lastTransactionID ):
    #print ("Statment: ",statement) 
    #print ("Trade: ", statement, trade) 

    # 1.1 - Open-Order - in das Depot eintragen (incl. Quantity)  
    if trade.openClose == "Open":
        opInsertOpenTrade ( dbCursor, statement, trade , lastTrade )
        opAmount = statement.amount # Wenn Option, dann haben wir schon einen Gewinn  
        return 0   
            
    elif trade.openClose == "Close":
        # Close-Order - 
        print ("+++ 282 ---- Hier kommt die Close-Order" , trade.transactionID , "close ", trade.openClose )
             
        #  2.1     suche die passende Open-Order
        openTransactionID = selectOpenTransactionID ( dbCursor, trade )
        # 23.06.23 - Wenn die Positionnicht drin ist, aufhören 
        if openTransactionID == None:
            print (f"+++ 282- keine Offene Position vorhanden, auch OK Open Transaction {openTransactionID} " )
            return 
        else:
            print (f"+++ 284 - Open Transaction {openTransactionID} " )
        openStatement = selectStatement ( dbCursor, openTransactionID)
        openTrade = selectTrade ( dbCursor, openTransactionID)
        # opOpenTrade = opSelectTrade (dbCursor, openTransactionID)

        #  2.3     Trage Trade in Closed ein
        cpInsertClosedTrade ( dbCursor, openStatement, statement, openTrade, trade ) #  opOpenTrade )

        # 2.3 Erzeuge für diesen Trade eien entsprechende Steuerdatensatz  
        # txInsertCloseTaxTrade ( dbCursor, openStatement, statement, openTrade, trade, quantity  ) # Close
        #
        
        #  2.4     Update die Quantity im Depot, falls 0 lösche den Eintrag im Depot 
        # ??? Klären, ob das stimmt  
        if (  trade.quantity >= -openTrade.quantity ):  # Schaen, ob es noch mehr zu tun gibt  
            opDeleteTrade ( dbCursor, openTrade)
        else:
            opUpdateTrade ( dbCursor, openTrade, openStatement, statement, trade )        
        
        return trade.quantity  - openTrade.quantity  # ??? noch klären, ob das stimmt 

    else :
        print (trade.openClose , " ist weder Close noch Open, da ist was faul")
        exit () 

#--------- Eintragen in das Depot -----------
# 1,1 - Open-Order - in das Depot eintragen (incl. Quantity)  
def opInsertOpenTrade ( dbCursor, statement, trade, lastTrade ): # lastTradeTime, lastTransactionID):
    opCombo = calculateCombo ( dbCursor , statement, trade, lastTrade )
    opAmount = statement.amount
    # Einzelpreis = Gesamtpreis/trade.quantity 
    daysToExpiration= calculateDaysToExpiration( trade ) 
    
    sql = "INSERT INTO openPositions (symbol, description, conid, amount, quantity, transactionID, buySell, assetCategory, combo , daysToExpiration) "
    sql += "Values ( ?, ?, ?, ?, ?, ?, ?, ?, ?, ? )"
    werte = ( trade.symbol, trade.description, trade.conid, opAmount , trade.quantity, trade.transactionID, trade.buySell, trade.assetCategory, opCombo, daysToExpiration)
    dbCursor.execute ( sql, werte )
    dbCursor.commit() # Wirklich eintragen 

    str = f"Open-Trade {trade.description} mit Gesamtpreis {opAmount} mit Anzahl: {trade.quantity} eingetragen , Combo: {opCombo}"
    print ( str )
    updateStatementOpInfo ( dbCursor, statement, str  ) 


def cpInsertClosedTrade ( bCursor, openStatement, closeStatement, openTrade, closeTrade  ) :  

    daysToExpiration= calculateDaysToExpiration( openTrade )   
    daysInTrade = calculateDaysInTrade ( openTrade, closeTrade ) 

    openAmount  = openStatement.amount
    closeAmount = closeStatement.amount

    openQuantity  = openTrade.quantity 
    closeQuantity = closeTrade.quantity        

    # Der Einzelpreis beim öffnen 
    closeResult = (openAmount/closeQuantity)*closeQuantity + closeAmount  
    
    comment = (f"{openTrade.assetCategory}  {openTrade.buySell} --> {closeTrade.buySell} {openAmount} {closeAmount} Ergebnis: {closeResult}")

    sql = "INSERT INTO closedPositions (transactionID, symbol, description, conid, assetCategory, " 
    sql += "openTransactionID, openBuySell, openDate, openAmount, daysToExpiration, openQuantity, "
    sql += "closeDate, daysInTrade, closeAmount, closeQuantity, closeBuySell, closeResult, comment ) "
    sql += "Values ( ?, ?, ?, ?, ?, "
    sql += "         ?, ?, ?, ?, ?, ?, "
    sql += "         ?, ?, ?, ?, ?, ?, ?)"
    werte = ( closeTrade.transactionID, closeTrade.symbol, closeTrade.description, closeTrade.conid,  closeTrade.assetCategory, \
              openTrade.transactionID, openTrade.buySell, openTrade.tradeDate, openAmount, daysToExpiration, openQuantity,   \
              closeTrade.tradeDate, daysInTrade, closeAmount , closeQuantity, closeTrade.buySell, closeResult, comment )
    dbCursor.execute ( sql, werte )
    dbCursor.commit() # Wirklich eintragen 

    str = f"Close-Trade {closeTrade.description} mit Gesamtpreis {closeAmount} mit Anzahl: {closeTrade.quantity} eingetragen "
    print (str)
    updateStatementOpInfo ( dbCursor, statement, str  ) 

# ------------- TAX ------------------------------------------

def txIsOptOpenTrade ( trade ):
    if trade.openClose == "Open" and trade.buySell == "SELL" and trade.assetCategory == "OPT" :
        return True
    else:
        return False

def txTryInsertTaxStatement ( dbCursor, statement, trade, datensatz) :
    # Bei einem Sell/Verkauf wird nur Fall einer Option eine Steuerdatensatz erzeugt
    # die Vereinnahme Prämie ist ein steuerlicher Gewinn 
    #
    # 07.07. Neuer Beschluss:... jedes Statment kommt in die Tax Tabelle, damit alles dokumentiert ist 

    info = f"Category: {trade.assetCategory} OpenClose: {trade.openClose}  buySell:  {trade.buySell} "
    print ( f"++ 333 - tx Insert Tax Statment {info}" )

    # if txIsOptOpenTrade ( trade ) :
    #    # Vereinnahmete Prämie ist beim Eröffnen steuerlich relevant
    #   txInsertTaxStatement ( dbCursor, statement, trade , datensatz)
    #  return 
    
    #if ( trade.openClose == "Close" ):
    #   # alle Close-Fälle verursachen einen steuerlichen Datensatz 
    #  txInsertTaxStatement ( dbCursor, statement, trade , datensatz )

    # 07.07.23 BG Jedes Statment, auch bei 0€ soll in die Steuertabelle 
    txInsertTaxStatement ( dbCursor, statement, trade , datensatz )     

def txInsertTaxStatement ( dbCursor, statement, trade, datensatz ) :
    # Datensatz: nur zum Testen, dass ich nachvollziehen kann, was passiert 
    # Bei einem verkauften Call ist der steuerliche Gewinn die Prämie  

    if trade.openClose == "Open":
        print (f"+++ TX Insert Im Zustand Open, Amount ist {statement.amount }  TradeOPen: {trade.openClose}")
        taxFiFoResult = statement.amount 
    else:
        print (f"+++ TX Insert Im Zustand Close, Amount ist {statement.amount }")
        taxFiFoResult = taxCalculateFiFoResult ( dbCursor, statement, trade )
    
    txYear = statement.datum.year

    # 06.07.23 BG Umgestellt auf Base-Amount 
    # 06.07.23 Bg ergänzt zur besseren Lesbarkeit der Tabelle: underlyingSymbol, activityDescription
    # 11.07.23 BG Feld - fxRateToBase + action weiter gereicht 
    comment = f"in: txInsertTax Statement {trade.openClose} {trade.description} mit Gesamtpreis {statement.amount} mit Anzahl: {trade.quantity} eingetragen FiFo {taxFiFoResult} "
    sql = "INSERT INTO taxStatements (transactionID, symbol, underlyingSymbol, conid, description, activityDescription, assetCategory, "
    sql += "openClose, tradeDate, taxYear, buySell, putCall, baseAmount, baseCurrency, baseBalance, fifoPnlRealized, " 
    sql += "taxFiFoResult, quantity, comment, datensatz, fxRateToBase, action ) "
    sql += "Values ( ?, ?, ?, ?, ?, ?, ?, "
    sql += "         ?, ?, ?, ?, ?, ?, ?, ? , ?, "
    sql += "         ?, ?, ?, ?, ? , ? )"
    werte = ( statement.transactionID, trade.symbol, statement.underlyingSymbol, trade.conid,  trade.description, statement.activityDescription, trade.assetCategory, \
              trade.openClose, trade.tradeDate, txYear, trade.buySell, trade.putCall, statement.amount, statement.IBcurrency, \
              statement.balance, trade.fifoPnlRealized, taxFiFoResult, trade.quantity, comment, datensatz, trade.fxRateToBase, trade.action )
    print ( sql ) 
    dbCursor.execute ( sql, werte )
    dbCursor.commit() # Wirklich eintragen 
    print (comment)
    updateStatementTxInfo ( dbCursor, statement, f"+ 530 - Tx eingtragen Abschluss: {taxFiFoResult} Laut fifoPnlRealized  {trade.fifoPnlRealized} Erfolg oder nicht?    " ) 
    txUpdateTaxStatement ( dbCursor , statement, trade, statement.transactionID, trade.fifoPnlRealized, taxFiFoResult) # Hier wird die Steuer zugeordnet 

# ---- TAX -  Hiwr ist die Steuerlogik , was wo hin kommt 
def txUpdateTaxStatement ( dbCursor , statement, trade, transactionID, fifoPnlRealized, taxFiFoResult ): # Hier wird die Steuer zugeordnet 

    info = f"Category: {trade.assetCategory} OpenClose: {trade.openClose}  buySell:  {trade.buySell} "
    print ( "+++ txUpadte Tax Statment" + info )
    sqlField = None

    if txIsOptOpenTrade ( trade ):  # STO = Sell to Open Stillhalter !! Steuerlich relevant 
        # Eine Verkauf eine Stillhalter-Position erzeugt eine Gewinn 
        sqlField = f"taxAmountStillhalterGewinn = {taxFiFoResult}" 
        
    elif trade.openClose == "Open" :
        return # In allen anderen Fällen bei Open gibt es steuerlich nix zu tun 
    
    # Wir sind im Zustand: CLOSE#
    # 11.07.23 Bg wir werden schlauer 
    elif trade.assetCategory == "OPT":  
        # 11.07. BG Ich habe das Feld:. . fifoPnLRealized, da sagt mir, ob der Trade erfolgreich 
        if trade.quantity > 0: # Ich kaufe eine Stillhalter-Option zurück. 
            if fifoPnlRealized > 0.0 : # 11.07. BG Trade war erfolkgreich 
                sqlField = f"taxAmountStillhalterGewinn = {taxFiFoResult}" 
            else: 
                sqlField = f"taxAmountStillhalterverlust = {taxFiFoResult}" 
        # eine Long-Position wurde verkauft        STC = Sell To Close  
        elif fifoPnlRealized > 0.0 : # Trade war erfolgreich eine Long-Position wurde verkauft 
            sqlField = f"taxAmountTerminGewinn = {taxFiFoResult}" 
        else: 
            sqlField = f"taxAmountTerminVerlust = {taxFiFoResult}" 

    # 11.07 - Was muss ich bei AKtien machen         
    elif trade.assetCategory == "STK":  
        if taxFiFoResult > 0.0 :
            sqlField = f"taxAmountAktienGewinn = {taxFiFoResult}" 
        else: 
            sqlField = f"taxAmountAktienverlust = {taxFiFoResult}" 

    if sqlField != None:
        sql = f"UPDATE taxStatements SET {sqlField} WHERE transactionID = {transactionID} "
        print ( f"{sql}" ) 
        dbCursor.execute ( sql)  
        dbCursor.commit() # Wirklich eintragen 

def taxCalculateFiFoResult ( dbCursor, statement, trade ): 
    # Ausgangsbasis: zu einem abgeschlossen Trade kann es mehr oder weniger offene Positionen geben
    # diese haben jeweils unterschiedliche Einkaufs/Verkaufspreise . Das muss ich entsprechend berücksichtigen.
    # So richtig gut klappt es auf einzelne Stücke, d.h. ich rechne für jedes Stück den Einkaufspreis 

    # Vorgehensweise: gehe die openPositions und addiere Schrittweise den Gewinn  
    #      
    
    
    
    if trade.assetCategory == "STK":
        # 11.07.23 Aktien richtig behandelt 
        return trade.fifoPnlRealized 
    elif trade.assetCategory == "OPT" and trade.action == "STC":
        # 13.07.23 BG / FR - Exit einer Long-Option 
        return trade.fifoPnlRealized
    else:
        return statement.amount

    #********************************* BG 05.07. ---  erst mal abgeklemmt --------
    opAnzahl = opGetPositionsCount (trade.conid)
    print (f" ---- Calculate Result - Anzahl Positionen >{opAnzahl}<")
    if opAnzahl == 0:
        # Keine offenen Positionen vorhanden , nehmen wir die IB-Angabe
        if ( statement.amount) == 0 :
            print ("+++++++++++ Houston - wir haben ein Problem, keinen Gewinn mit der Position")
            return ( 0.0) 
        else:
            print (f"Bei {statement.description} gibt es keine offenen Position, Amount {statement.amount} aus Statment verwendet")
            return ( statement.amount )

    quantity =  abs(trade.quantity) # wir wollen die Absolute  
    amountSingle = statement.amount / quantity #
    print ( f"fiFo: Ausgangsbasis Coinid {trade.conid}  Gesamtpreis {statement.amount} Einzelpreis {amountSingle}")  
    
    fifoResult = Decimal (0.0) # Ausgangsbasis

    sql = f"SELECT * FROM openPositions WHERE conid = {trade.conid} ORDER BY transactionID"
    print ( sql )
    dbCursor.execute ( sql) 
    for op in dbCursor.fetchall ():
        # Alle offenen Positionen abarbeiten, bis wir die Quantity abgearbeit haben 
        opQuantity = op.quantity 
        opAmountSingle = op.opAmount / abs(opQuantity)
        print (f"Start: AnzahlGesamt: {trade.quantity} Open-Gesamt: {op.opAmount} Close-Gesamt: {statement.amount}")
        for i in range (abs(opQuantity)):
            singleResult = amountSingle + opAmountSingle 
            fifoResult += singleResult
            quantity -= 1 # Einen haben wir abgearbeitet 
            print (f"AnzahlGesamt: {trade.quantity} Noch zu berechnen: {quantity} Open: {opAmountSingle} Close: {amountSingle} EinzelResult {singleResult} Bisherige Summe {fifoResult}")
            if quantity == 0 :
                print (f"IB in $: {trade.fifoPnlRealized} Deutsche Steuer Ergebnis in € {fifoResult}")
                return fifoResult 

# --------- Start des Programms --------------------------

#Datenbank öffnen 
# Verbindung zur Access-Datenbank herstellen
DBdatenbank = ".\ib.accdb"
DBdriver="{Microsoft Access Driver (*.mdb, *.accdb)}"
print ("Datenbank: ", DBdriver , " ", DBdatenbank )
#dbConn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=", datenbank, ";')
#dbConn = pyodbc.connect(r'Driver=', DBdriver,';DBQ=', DBdatenbank, ';')
# Das andere Wollte nicht 

dbConn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=.\ib.accdb;')

# Cursor-Objekt erstellen
dbCursor = dbConn.cursor()

#alle Daten löschen 
print ("Vor Löschen OP")
sql= "DELETE FROM openPositions"
dbCursor.execute (sql)
# Closed Closed
print ("Vor Löschen CP")
sql= "DELETE FROM closedPositions"
dbCursor.execute (sql)
sql= "DELETE FROM taxStatements"
dbCursor.execute (sql)

# Nur zum Steuern des Programms 
# Zeile 495 
i = 0
datensatz = 0 
anfang = 1 #  Hier kann ich auch sagen: ... starte erst bei Datensatz x 
max = 2000

#sql = "SELECT * FROM statements, trades WHERE statement.TransactionID = orders.TransactionID"
# 17.06. Versuch, mit einem Join das zu machen, ist gescheitert, stört aber nicht, so habe ich zwei einzelne Select gemacht 
# sql += " AND underlyingSymbol = 'NVDA' "0 
# sql += " AND description = 'NVDA 20JAN23 177.5 C' "
# sql += " AND description = 'NVIDIA CORP' " 
# sql = "SELECT * FROM statements WHERE tradeID > 0 ORDER BY transactionID, levelOfDetails"

desc = "FIRST SOLAR INC"
desc = "NVIDIA CORP"
#desc = "NVDA 17FEB23 177.5 P"
desc = "NVDA 20Jan23 177.5 C"

sql = "SELECT * FROM statements WHERE tradeID = tradeID "
#sql += f" AND description = '{desc}' " # Versuch, die Aktien zu klären 
sql += " ORDER BY transactionID, levelOfDetail"


print (f"Start der Eintragung {sql}"  )
dbCursor.execute ( sql) 

#lastTradeTime = " " # Letzten Tradetime merken
#lastTransactionID = 0
lastTrade = None

baseCurrency = "BaseCurrency"

print ( f"Ausgangslage: Test mit {desc} in der Range {anfang} und {max}")

for statement in dbCursor.fetchall () :

    # Nur zu Testzwecken, um nur einige Datensätze durchzugehen  
    i += 1
    datensatz = i 
    print (" ")
    print (f"+ 493 - ++++++++++ Datensatz {i} ++ID: {statement.id} ++++  {statement.activityDescription} +++++ Max = {max}++++ Start {anfang}+++++")
    if (i > max): 
        print (f"+ 495 - +++++++Abbruch wegen max {max}")
        break # Testweise vorher aufhören

    if (i < anfang ):
        print (f"+ 497 - Überspringe Datensatz {i} (zum Testen) Anfang {anfang}")
        continue   # 02.07. die ersten Datensätze überspringen 
    
    # Noch zu bearbeiten 
    #qifInsertTransaction ( statement) # 09.07.2023 Money mit eingeführt 
    
    trade = selectTrade ( dbCursor, statement.transactionID ) 
    
    print (f"+++ 496 - Statement {statement}")
    
    # 05.07.23 BG umgestellt auf "levelOfDetail" 
    if ( statement.levelOfDetail != baseCurrency ):
        # 06.07.23 BG Aus dem zweiten Datensatz ergänze ich den amount / currency 
        updateTxAmountCurrency ( dbCursor, statement )
        print (f"+ - 560 Update Currency:  {statement.IBcurrency} Amount: {statement.amount} Balance: {statement.balance} Beschreibung: {statement.activityDescription}")
        continue 

    # 28.06.23 BG Klären, warum es keinen Trade zu dieser Transaktion gibt 
    if trade is None:
        str = f"+ - 500 - Keinen Trade gefunden für {statement.activityDescription} ID: {statement.transactionID}"
        print (str)
        updateStatementTxInfo ( dbCursor, statement, str ) 
        continue 
    else :
        print (f"+++ 505 - Trade {trade} zu Statment gefunden ")
    
    # 01.07.2023 - Wenn ich mitten drin starte, Skippen, wenn ich keien offenen Positionen habe 
    anzahl = opGetPositionsCount (dbCursor, trade.conid) # Klären, ob wir genug Quantity für diese conid haben 
    print (f"+ 517 opGetPositionsCount (trade.conid) > Trade Quantity = {trade.quantity}  Anzahl Offene Positionen: {anzahl} - Welcher Typ: ? {trade.openClose}  ")
    # 02.07. Quantity muss abs()  sein 
    
    ################## !!!!!!!!!!!!!!!!!!!!!
    # 06.07.23 BG - Aktuell wollen wir nur Steuer haben - deshalb achten wir zur Zeit nicht,ob es in den offenen Positionen ist  
    if trade.openClose == "Close" and abs(trade.quantity) > anzahl:
        str = f"+ - 508 - Close + Anzahl > 0 , Anzahl {trade.quantity} , Denncohweiter Frage: ? nicht in den offenen Positionen {anzahl} enthalten" 
        print (str)
        # Wir teilen es zumidnest mit, machen aber weiter 
    #    updateStatementTxInfo ( dbCursor, statement, str )
    #    continue

    print ("# - 513 - Insert Tax Statment ")
    # Klären, ob es einen Tax-Datensatz geben soll, wenn ja, wird er eingetragen 
    txTryInsertTaxStatement ( dbCursor, statement, trade , datensatz ) # Jedes Statement gibt einen Steuerdatensatz, weil der ist correct
      
    # Der Teil für IB-Info 
    updateOpenPositions ( dbCursor , statement,trade, lastTrade ) # OLD: lastTradeTime, lastTransactionID )
    lastTrade = trade 

    # Zum Testen 
 
print (f" FERTIG mit {datensatz} Datensätzen" ) 

dbCursor.commit()
dbConn.close()

# 09.07.23 BG 
#qif.to_qif(".\money.qif")  


    
